Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Load the Data for Delhi

In [2]:
neighborhoods = pd.read_csv("Delhi_Spatial2.csv")
print('Data Loaded')

Data Loaded


In [3]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,South Delhi,Aliganj,28.098882,80.578288
1,North East Delhi,Maujpur,28.288693,77.435331
2,North East Delhi,Dayalpur,28.302218,77.388417
3,North West Delhi,Onkar Nagar,28.361667,76.965915
4,South Delhi,Hamdard Nagar,28.377701,76.918557


And make sure that the dataset has all 6 boroughs and 426 neighborhoods.

In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 9 boroughs and 195 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [5]:
address = 'Delhi City'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi City are 28.6517178, 77.2219388.


#### Create a map of Delhi City with neighborhoods superimposed on top.

In [6]:
# create map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  
    
map_delhi

In [7]:
CLIENT_ID = 'SZ2220HV3ECG4QJUTHW0NSRLVO0CD54JNAMB3I4OF5RX0MXN' # your Foursquare ID
CLIENT_SECRET = '2WPQ0ZSOWJWOFC1I5HBHMDO2KAQ443QBKUX3BJ1N0S13QSFM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SZ2220HV3ECG4QJUTHW0NSRLVO0CD54JNAMB3I4OF5RX0MXN
CLIENT_SECRET:2WPQ0ZSOWJWOFC1I5HBHMDO2KAQ443QBKUX3BJ1N0S13QSFM


From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## 2. Explore Neighborhoods

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [10]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Aliganj
Maujpur
Dayalpur
Onkar Nagar
Hamdard Nagar
Vivek Vihar
Chattarpur
Jaunapur
Arjungarh
Deoli
Satbari
Ignou
IGNOU New Delhi
Maidan Garhi
Badarpur
Talimabad
Molarband
Ghitorni
BTPS
Tugalkabad
Jaitpur
Mehrauli
Okhla
A F Rajokari
Saket
Chittranjan Park
Dakshinpuri
Vasant Kunj
Rajokari
Kapashera
Pushp Vihar
Lado Sarai
Govindpuri
Malviya Nagar
Jhatikara
Badusarai
Bijwasan
Kalkaji
Samalkha
Greater Kailash
Mahipalpur
Shahbad Mohammadpur
JNU
Dhansa
Hauz Khas
Munirka
Sukhdev Vihar
Vasant Vihar
Raota
New Friends Colony
IGI Airport
Sriniwaspuri
Ujwa
Andrewsganj
East Of Kailash Phase
Rewla Khanpur
Nauroji Nagar
R K Puram
South Extn
R K Puram
Okhla Industrial Estate
Bikaji-cama place
Lajpat Nagar
South Delhi Campus
Papravat
Hari Nagar Ashram
Jungpura
Sarojini Nagar
Palam Airport
Dwarka Sector-6
Pandara Road
Lodi Colony
Kakrola
Udyog Bhawan
Nangal Raya
Dargah Sharif
Hazrat Nizamuddin
Sagarpur
Malcha Marg
Najafgarh
Rashtrapati Bhawan
Rohtash Nagar
Mayur Vihar
Dhaula Kuan
Dabri
Matiala
Patparganj

In [11]:
neighborhoods_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Onkar Nagar,28.361667,76.965915,Axis Bank ATM,28.358549,76.975128,ATM
1,Vivek Vihar,28.438785,77.046471,Costa Coffee,28.439859,77.044103,Coffee Shop
2,Vivek Vihar,28.438785,77.046471,Domino's Pizza,28.443123,77.055295,Pizza Place
3,Vivek Vihar,28.438785,77.046471,Costa Coffee,28.440988,77.056230,Coffee Shop
4,Vivek Vihar,28.438785,77.046471,"KFC, Medanta",28.439186,77.042077,Fried Chicken Joint


In [12]:
neighborhoods_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A F Rajokari,1,1,1,1,1,1
Anandvas Shakurpur,5,5,5,5,5,5
Andrewsganj,38,38,38,38,38,38
Arjungarh,3,3,3,3,3,3
Babarpur,8,8,8,8,8,8
Badarpur,5,5,5,5,5,5
Badli,2,2,2,2,2,2
Bakkarwala,1,1,1,1,1,1
Bakoli,2,2,2,2,2,2


In [13]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 203 uniques categories.


## 3. Analyze Each Neighborhood

In [14]:
# one hot encoding
delhi_onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Botanical Garden,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Internet Cafe,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Kebab Restaurant,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Rental Car Location,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Stables,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Women's Store
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Onkar Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Vivek Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [15]:
delhi_onehot.shape

(2179, 203)

In [17]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()

In [18]:
delhi_grouped.shape

(163, 203)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
num_top_venues = 5

for hood in delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = delhi_grouped[delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A F Rajokari----
                 venue  freq
0  Sporting Goods Shop   1.0
1          Yoga Studio   0.0
2               Mosque   0.0
3      Motorcycle Shop   0.0
4        Movie Theater   0.0


----Anandvas Shakurpur----
                  venue  freq
0  Fast Food Restaurant   0.2
1                 Diner   0.2
2           Pizza Place   0.2
3           Snack Place   0.2
4                Market   0.2


----Andrewsganj----
                venue  freq
0   Indian Restaurant  0.16
1  Italian Restaurant  0.11
2                Café  0.11
3         Pizza Place  0.08
4              Market  0.05


----Arjungarh----
                        venue  freq
0  Tourist Information Center  0.67
1           Indian Restaurant  0.33
2                 Yoga Studio  0.00
3     North Indian Restaurant  0.00
4               Movie Theater  0.00


----Babarpur----
               venue  freq
0               Park  0.25
1    Motorcycle Shop  0.12
2        Men's Store  0.12
3     Clothing Store  0.12
4  Electronics S

#### Let's put that into a *pandas* dataframe


First, let's write a function to sort the venues in descending order.

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A F Rajokari,Sporting Goods Shop,Women's Store,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
1,Anandvas Shakurpur,Pizza Place,Diner,Fast Food Restaurant,Market,Snack Place,Dim Sum Restaurant,Falafel Restaurant,French Restaurant,Food Truck,Food Court
2,Andrewsganj,Indian Restaurant,Italian Restaurant,Café,Pizza Place,Market,Hotel,Performing Arts Venue,Nightclub,Convenience Store,Park
3,Arjungarh,Tourist Information Center,Indian Restaurant,Women's Store,Event Service,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
4,Babarpur,Park,Electronics Store,ATM,Motorcycle Shop,Ice Cream Shop,Clothing Store,Men's Store,Fabric Shop,Food Court,Food & Drink Shop


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [22]:

# set number of clusters
kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 0, 2, 1, 4, 4], dtype=int32)

In [72]:
# add clustering labels
#neighborhoods_venues_sorted.drop('Cluster Labels1',1,inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

delhi_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
delhi_merged = delhi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
delhi_merged.head() # check the last columns!

In [76]:
#drop rows having missing value
delhi_merged.dropna(subset=['Cluster Labels'], axis=0, inplace=True)
delhi_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North West Delhi,Onkar Nagar,28.361667,76.965915,1.0,ATM,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
5,East Delhi,Vivek Vihar,28.438785,77.046471,4.0,Coffee Shop,Pizza Place,Convenience Store,Fast Food Restaurant,Fried Chicken Joint,Event Space,Food Truck,Food Court,Food & Drink Shop,Food
6,South West Delhi,Chattarpur,28.456200,77.182350,2.0,Hotel,Resort,Hotel Bar,Electronics Store,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farm
7,South West Delhi,Jaunapur,28.462455,77.154260,4.0,Bar,Women's Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
8,South West Delhi,Arjungarh,28.468200,77.132300,4.0,Tourist Information Center,Indian Restaurant,Women's Store,Event Service,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant


In [77]:
#Change Datatype of Cluster Labels filed to int
delhi_merged = delhi_merged.astype({'Cluster Labels':int}) 
delhi_merged.head()


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North West Delhi,Onkar Nagar,28.361667,76.965915,1,ATM,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
5,East Delhi,Vivek Vihar,28.438785,77.046471,4,Coffee Shop,Pizza Place,Convenience Store,Fast Food Restaurant,Fried Chicken Joint,Event Space,Food Truck,Food Court,Food & Drink Shop,Food
6,South West Delhi,Chattarpur,28.456200,77.182350,2,Hotel,Resort,Hotel Bar,Electronics Store,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farm
7,South West Delhi,Jaunapur,28.462455,77.154260,4,Bar,Women's Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
8,South West Delhi,Arjungarh,28.468200,77.132300,4,Tourist Information Center,Indian Restaurant,Women's Store,Event Service,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant


In [78]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(delhi_merged['Latitude'], delhi_merged['Longitude'], delhi_merged['Neighborhood'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [79]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Badarpur,Indian Restaurant,Train Station,Light Rail Station,Event Space,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
125,Karol Bagh,Business Service,Convenience Store,Park,Train Station,Women's Store,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
127,Geeta Colony,Dessert Shop,Train Station,Burger Joint,Women's Store,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
130,Zakhira,Indian Restaurant,Department Store,Train Station,Fast Food Restaurant,Women's Store,Event Space,Food Truck,Food Court,Food & Drink Shop,Food
131,Sarai Rohilla,Train Station,Pizza Place,Hotel,Light Rail Station,Dim Sum Restaurant,Event Space,Food Truck,Food Court,Food & Drink Shop,Food
132,Jwalaheri,Pizza Place,Train Station,Sandwich Place,Bar,Bakery,Asian Restaurant,Indian Restaurant,ATM,Ice Cream Shop,Hotel Bar
134,Seelampur,Platform,Shopping Mall,Train Station,Electronics Store,Light Rail Station,Event Space,Food Truck,Food Court,Food & Drink Shop,Food
135,Punjabi Bagh,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Convenience Store,Italian Restaurant,Train Station,Pizza Place,Fast Food Restaurant,Event Space,Fabric Shop,Falafel Restaurant
136,Inderlok,Pizza Place,Indian Restaurant,Train Station,Hotel,Fast Food Restaurant,Event Service,Food Truck,Food Court,Food & Drink Shop,Food
139,Yamuna Vihar,Platform,Shopping Mall,Train Station,Hotel,Event Service,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant


#### Cluster 2

In [80]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Onkar Nagar,ATM,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
11,Ignou,ATM,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
12,IGNOU New Delhi,ATM,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
36,Bijwasan,ATM,Gym / Fitness Center,Women's Store,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
79,Najafgarh,ATM,Motorcycle Shop,Men's Store,Women's Store,Falafel Restaurant,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
129,Bakkarwala,ATM,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
156,Sultanpuri,ATM,Buffet,Women's Store,Falafel Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
162,Johripur,ATM,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market


#### Cluster 3

In [81]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Chattarpur,Hotel,Resort,Hotel Bar,Electronics Store,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farm
10,Satbari,Hotel,ATM,Women's Store,Event Space,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
28,Rajokari,Hotel,Restaurant,Farm,Event Service,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
29,Kapashera,Hotel,Water Park,Food Truck,Food Court,Women's Store,Event Service,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
38,Samalkha,Hotel,Indian Restaurant,Hotel Bar,American Restaurant,Spa,Rental Car Location,Vegetarian / Vegan Restaurant,Food,Food & Drink Shop,Event Service
40,Mahipalpur,Hotel,Hotel Bar,Spa,Indian Restaurant,Italian Restaurant,Light Rail Station,Diner,North Indian Restaurant,Plaza,Punjabi Restaurant
90,Daulatpur,Hotel,Snack Place,Café,Event Space,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
91,Naraina Village,Hotel,Snack Place,Café,Arcade,Multiplex,Sandwich Place,Fast Food Restaurant,Indian Restaurant,Pizza Place,Event Service
92,Naraina Industrial Estate,Hotel,Snack Place,Café,Arcade,Multiplex,Sandwich Place,Fast Food Restaurant,Indian Restaurant,Pizza Place,Event Service
115,Pahar Ganj,Hotel,Indian Restaurant,Pizza Place,Café,Gift Shop,Hostel,Restaurant,Platform,Bar,Bakery


#### Cluster 4

In [82]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 3, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Dargah Sharif,Botanical Garden,Women's Store,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
76,Hazrat Nizamuddin,Botanical Garden,Women's Store,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant


#### Cluster 5

In [83]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 4, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Vivek Vihar,Coffee Shop,Pizza Place,Convenience Store,Fast Food Restaurant,Fried Chicken Joint,Event Space,Food Truck,Food Court,Food & Drink Shop,Food
7,Jaunapur,Bar,Women's Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
8,Arjungarh,Tourist Information Center,Indian Restaurant,Women's Store,Event Service,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
9,Deoli,Arts & Crafts Store,Women's Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
13,Maidan Garhi,Athletics & Sports,Business Service,Women's Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
15,Talimabad,Furniture / Home Store,Historic Site,Dessert Shop,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
17,Ghitorni,Clothing Store,Women's Store,Event Space,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
19,Tugalkabad,Business Service,ATM,Stadium,Castle,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
20,Jaitpur,Clothing Store,Women's Store,Event Space,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
21,Mehrauli,Thai Restaurant,Metro Station,Market,Sculpture Garden,Arts & Crafts Store,Public Art,Train Station,Flea Market,Japanese Restaurant,Light Rail Station
